<a href="https://colab.research.google.com/github/EduardoLawson1/Data_Augmentation/blob/main/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
from tqdm import tqdm
from glob import glob
from albumentations import CenterCrop, RandomRotate90,  CLAHE, HorizontalFlip
from albumentations.augmentations.geometric.transforms import F

In [ ]:
def load_data(path):
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))
    return images, masks

def create_dir(path):  #função para criar diretórios
    if not os.path.exists(path):
      os.makedirs(path)

def augment_data(images, masks, save_path, augment=True):
    H = 256 # Altura da imagem
    W = 256 #largura

    for x, y in tqdm(zip(images, masks), total=len(images)):
        name = x.split("/")[-1].split(".")
        image_name = name[0]
        image_extn = name[1]

        name = y.split("/")[-1].split(".")
        mask_name = name[0]
        mask_extn = name[1]

        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)
        #print(x.shape, y.shape)

        if augment == True:
            aug = CenterCrop(128, 128, p = 0.2)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = RandomRotate90(p = 0.6)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]

            aug = HorizontalFlip( p = 0.6)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            aug = CLAHE( p = 0.6)
            augmented = aug(image=x, mask=y)
            x4 = augmented["image"]
            y4 = augmented["mask"]



            save_images = [x, x1, x2, x3, x4]
            save_masks = [y, y1, y2, y3, y4]
        else:
            save_images = [x]
            save_masks = [y]

        idx = 0
        for i, m in zip(save_images, save_masks):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            tmp_img_name = f"{image_name}_{idx}.{image_extn}"
            tmp_msk_name = f"{mask_name}_{idx}.{mask_extn}"

            image_path = os.path.join(save_path, "images", tmp_img_name)
            mask_path = os.path.join(save_path, "masks", tmp_msk_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)


if __name__ == "__main__":
    path = "/content/drive/MyDrive/pré_data_augmentation"
    images, masks = load_data(path)
    print(f"Original images: {len(images)} - Original masks: {len(masks)}")

    create_dir("/content/drive/MyDrive/new_data/images/")
    create_dir("/content/drive/MyDrive/new_data/masks/")

    save_path = "/content/drive/MyDrive/new_data"
    augment_data(images, masks, save_path, augment=True)

    images, masks = load_data(save_path)
    print(f"Augmented images: {len(images)} - Augmented masks: {len(masks)}")

